In [2]:
import pandas as pd
import numpy as np
import time
from scipy.sparse import coo_matrix
from lightfm import LightFM 
from lightfm.data import Dataset as LFM_Dataset
from sklearn.preprocessing import LabelEncoder
from lightfm.evaluation import precision_at_k
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader
from surprise import KNNWithMeans
from surprise.model_selection import train_test_split
from scipy.stats import rankdata
from surprise import SVD


In [3]:
# Загрузка датасета
daily_sales_df = pd.read_csv("/kaggle/input/daily-sales-352-df/daily_sales_352_df.csv")

In [51]:
daily_sales_df 

,item,user,rating,timestamp,description,title,date
0,B0000CFH7B,A3SJ6OST09HD86,1.0,1309737600,['Wilton-color mist food color spray. This eas...,"Wilton 710-5506 Color Mist Spray 1.5oz, Black",2011-07-04
1,B0000CFH7B,A3SJ6OST09HD86,1.0,1309737600,['Wilton-color mist food color spray. This eas...,"Wilton 710-5506 Color Mist Spray 1.5oz, Black",2011-07-04
2,B0000CFH7B,A3I7WIYAQMJNTF,1.0,1305936000,['Wilton-color mist food color spray. This eas...,"Wilton 710-5506 Color Mist Spray 1.5oz, Black",2011-05-21
3,B0000CFH7B,A3I7WIYAQMJNTF,1.0,1305936000,['Wilton-color mist food color spray. This eas...,"Wilton 710-5506 Color Mist Spray 1.5oz, Black",2011-05-21
4,B0000CFH7B,A3C5231O0WQX10,5.0,1296000000,['Wilton-color mist food color spray. This eas...,"Wilton 710-5506 Color Mist Spray 1.5oz, Black",2011-01-26
...,...,...,...,...,...,...,...
818605,B01GH3JVZI,A2UPCGGZ0FU3XZ,5.0,1534982400,['The ViSalus Vi-Shape nutritional shake mix i...,ViSalus Nutritional Meal Replacement Shake Mix...,2018-08-23
818606,B01GH3JVZI,A1MJ18Y373RTFE,1.0,1534636800,['The ViSalus Vi-Shape nutritional shake mix i...,ViSalus Nutritional Meal Replacement Shake Mix...,2018-08-19
818607,B01GH3JVZI,A2KZ6WK0HWB2TD,5.0,1534464000,['The ViSalus Vi-Shape nutritional shake mix i...,ViSalus Nutritional Meal Replacement Shake Mix...,2018-08-17
818608,B01GH3JVZI,A3UBTA2DJ27IWF,2.0,1534464000,['The ViSalus Vi-Shape nutritional shake mix i...,ViSalus Nutritional Meal Replacement Shake Mix...,2018-08-17


In [5]:
# Загрузка данных о множителях
multiplier_2002_full_df = pd.read_csv("/kaggle/input/multiplier-2002-full-df/multiplier_2002_full_df.csv")

In [6]:
multiplier_2002_full_df

,product_id,multiplier
0,B0001LO3FG,0.975918
1,B00BUKL666,0.731228
2,B00542YXFW,0.538453
3,B008QMX2SG,0.632924
4,B00D3M2QP4,0.571657
...,...,...
1997,B01G4I8VGQ,0.207146
1998,B01F563CX0,0.472329
1999,B014THN4OI,0.095767
2000,B01ATWI17W,0.109778


# --- Item-based LightFM ---

In [7]:
# Создание набора данных LightFM
dataset = LFM_Dataset()
# Приспособление набора данных с уникальными пользователями и элементами
dataset.fit(users=daily_sales_df['user'].unique(), items=daily_sales_df['item'].unique())

In [8]:
# Создание матрицы взаимодействий на основе данных о продажах
interactions, weights = dataset.build_interactions(zip(daily_sales_df['user'], daily_sales_df['item']))

In [9]:
# Создание и настройка модели LightFM с заданными параметрами
model = LightFM(loss='warp', no_components=30, learning_rate=0.1)
epochs = 10
num_threads = 16

# Вручную контролируем процесс обучения и выводим прогресс
for epoch in range(epochs):
    start_time = time.time()
    model.fit_partial(interactions, num_threads=num_threads)
    epoch_duration = time.time() - start_time

    train_precision = precision_at_k(model, interactions, k=5).mean()

    print(f"Epoch {epoch + 1}/{epochs} | Train precision: {train_precision:.4f} | Epoch duration: {epoch_duration:.2f}s")

Epoch 1/10 | Train precision: 0.1267 | Epoch duration: 1.23s
Epoch 2/10 | Train precision: 0.2386 | Epoch duration: 0.79s
Epoch 3/10 | Train precision: 0.2461 | Epoch duration: 0.58s
Epoch 4/10 | Train precision: 0.2498 | Epoch duration: 0.57s
Epoch 5/10 | Train precision: 0.2521 | Epoch duration: 0.58s
Epoch 6/10 | Train precision: 0.2538 | Epoch duration: 0.58s
Epoch 7/10 | Train precision: 0.2550 | Epoch duration: 0.62s
Epoch 8/10 | Train precision: 0.2559 | Epoch duration: 0.56s
Epoch 9/10 | Train precision: 0.2566 | Epoch duration: 0.56s
Epoch 10/10 | Train precision: 0.2572 | Epoch duration: 0.56s


In [10]:
# Функция для получения k наиболее похожих элементов с использованием модели LightFM
def get_similar_items(model, item_id, k):
    item_idx = dataset.mapping()[2][item_id]    # Получение индекса элемента в матрице
    similarities = cosine_similarity(model.item_embeddings)[item_idx]    # Вычисление косинусного сходства между элементами

    # Найдите k наиболее похожих товаров
    most_similar_items = np.argsort(-similarities)[:k+1]

    # Получение обратного отображения для индексов элементов
    item_id_reverse_mapping = {v: k for k, v in dataset.mapping()[2].items()}

    # Получение идентификаторов k наиболее похожих элементов, исключая исходный элемент
    most_similar_item_ids = [item_id_reverse_mapping[item_idx] for item_idx in most_similar_items if item_id_reverse_mapping[item_idx] != item_id]

    return most_similar_item_ids[:k]

#  Item-based Surprise 

In [11]:
# Обработка данных для использования с библиотекой Surprise
# Рассчитываем количество покупок для каждого пользователя и товара
user_product_count = daily_sales_df.groupby(['user', 'item']).size().reset_index(name='purchase_count')

# Нормализация количества покупок для получения рейтинга (например, от 1 до 5)
user_product_count['rating'] = daily_sales_df['rating']

# Создание нового датафрейма, содержащего только нужные столбцы
ratings_df = user_product_count[['user', 'item', 'rating']]

# Загрузка данных в surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_df, reader)

# Разделение датасета на обучающую и тестовую выборки
trainset, testset = train_test_split(data, test_size=0.2)

# Создание и обучение модели 
sim_options = {
    'name': 'cosine',  # Выбор метрики сходства
    'user_based': False  # Item-based модель
}
algo = KNNWithMeans(sim_options=sim_options)
# algo = SVD(n_factors=650, n_epochs=450, lr_all=0.011, reg_all=0.02) # ушла в бесконечный цикл
algo.fit(trainset)

# Предсказания для тестовой выборки
predictions = algo.test(testset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [12]:
# Функция для получения рекомендаций товаров для пользователя с использованием модели Surprise
def get_recommendations(user_id, daily_sales_df, algo, n_recommendations=5):
    # Получаем список уникальных товаров
    unique_products = daily_sales_df['item'].unique()

    # Получаем список товаров, которые пользователь уже купил
    purchased_products = daily_sales_df.loc[daily_sales_df['user'] == user_id, 'item'].unique()

    # Список товаров, которые пользователь еще не купил
    not_purchased_products = set(unique_products) - set(purchased_products)

    # Предсказываем рейтинги для всех товаров, которые пользователь еще не купил
    predicted_ratings = [(product_id, algo.predict(user_id, product_id).est) for product_id in not_purchased_products]

    # Сортируем товары по убыванию предсказанных рейтингов и выбираем первые N товаров
    recommendations = sorted(predicted_ratings, key=lambda x: x[1], reverse=True)[:n_recommendations]

    # Возвращаем список рекомендованных товаров
    return [product_id for product_id, _ in recommendations]

In [13]:
# Функция для получения рекомендаций товаров для пользователя с использованием модели LightFM
def get_recommendations_lightfm(model, user_id, k=10):
    user_idx = dataset.mapping()[0][user_id]
    scores = model.predict(user_idx, np.arange(interactions.shape[1]))
    top_items = np.argsort(-scores)[:k]

    # Получите обратное отображение для индексов товаров
    item_id_reverse_mapping = {v: k for k, v in dataset.mapping()[2].items()}

    recommended_item_ids = [item_id_reverse_mapping[item_idx] for item_idx in top_items]

    return recommended_item_ids

# Функция для получения рекомендаций товаров с использованием ансамбля моделей LightFM и Surprise
def ensemble_recommendations(user_id, daily_sales_df, lightfm_model, surprise_algo, k=5):
    lightfm_recs = get_recommendations_lightfm(lightfm_model, user_id, k)
    surprise_recs = get_recommendations(user_id, daily_sales_df, surprise_algo, k)

    # Объединение рекомендаций от обеих моделей
    combined_recs = lightfm_recs + surprise_recs #['B00XA8XWGS', 'B000F4DKAI', 'B0001LO3FG', 'B00XABKAWY', 'B0014EOU4S']
    
    lin = []
    for i in combined_recs:
        lin.append(daily_sales_df.loc[daily_sales_df['item'] == i]['title'].iloc[-1])
        
    # Получение списка товаров, купленных пользователем
    user_purchase = daily_sales_df.loc[daily_sales_df['user'] == user_id]['title'].unique().tolist()
    
    # Удаление из списка рекомендаций товаров, которые пользователь уже купил
    result = list(filter(lambda x: x not in user_purchase, lin))
    
    # Возвращаем топ-N рекомендаций
    return result[:k]

### Рекомендачии товаров на основе подобия пользоваетля

In [14]:
# Пример использования функции ансамбля
user_id = "A2PF64RBR1G1SZ"  
k = 5  # Количество рекомендованных товаров

In [45]:
user_recommendations = ensemble_recommendations(user_id, daily_sales_df, model, algo, k=5)

print(f"Ансамбль рекомендаций для пользователя купившего следующие товары:\n") 
print('\n'.join(daily_sales_df.loc[daily_sales_df['user'] == user_id]['title'].unique().tolist()))

Ансамбль рекомендаций для пользователя купившего следующие товары:

Twinings of London Pure Camomile Herbal Tea Bags, 20 Count
Yogi Tea - Peach DeTox - Healthy Warming Blend - 6 Pack, 96 Tea Bags Total
Twinings of London English Afternoon Black Tea Bags, 20 Count (Pack of 6)
Jack Link's Beef Jerky, Sweet &amp; Hot, 1.8-Ounce Packages (Pack of 8)
Davidson's Tea Irish Breakfast Tea, 100-Count Tea Bags
Bragg Organic Unfiltered Apple Cider Vinegar, Raw, 16 Ounce - 1 Pack
V8 Original 100% Vegetable Juice, 5.5 oz. Can (8 packs of 6, Total of 48)
Glycerin Vegetable - 1 Quart (43 oz.) - Non GMO - Sustainable Palm Based - USP - KOSHER - PURE - Pharmaceutical Grade
96-Count Beantown Roasters Variety Pack Sampler of 12 Assorted Roasted Coffees, single-cup coffee pack sampler for Keurig K-Cup Brewers
Twinings of London Christmas Black Tea Bags, 20 Count
Planters Mixed Nuts, Lightly Salted Deluxe Mixed Nuts, 15.25 Ounce
Stacy's Parmesan Garlic &amp; Herb Flavored Pita Chips, 7.33 Ounce (Pack of 12)

In [46]:
user_recommendations

['Perrier Lime Flavored Carbonated Mineral Water, 16.9 fl oz. Plastic Bottles (24 Count)',
 'ANCIENT ORGANICS 100% Organic Ghee from Grass-fed Cows, 32oz',
 'Garden of Life Meal Replacement - Raw Organic Fit Powder, Original - High Protein for Weight Loss (28g) plus Fiber &amp; Probiotics, Organic &amp; Non-GMO Vegan Nutritional Shake, Packets (10 Count Tray)',
 'Little Stinker The Original Bag of Unicorn Farts Cotton Candy Funny Novelty Gift for Unique Birthday Gag Gift for Friends, Mom, Dad, Girl, Boy Grandson Stocking Stuffer While Elephant Christmas',
 'Organic Wise Ceylon Cinnamon Ground Powder, 1 lb-From a USDA Certified Organic Farm and Packed In The USA']

# Рекомендачии товаров на основе подобия товара

In [18]:
# Функция для получения похожих товаров с использованием модели Surprise
def get_similar_items_surprise(item_id, daily_sales_df, algo, k=5):
    # Получение уникальных пользователей и пользователей, которые купили данный товар
    unique_users = daily_sales_df['user'].unique()
    users_bought_item = daily_sales_df.loc[daily_sales_df['item'] == item_id, 'user'].unique()
    
    # Вычисление пользователей, которые не купили данный товар
    users_not_buy_item = set(unique_users) - set(users_bought_item)

    # Предсказание оценок для пользователей, которые не купили данный товар
    predicted_ratings = [(user_id, algo.predict(user_id, item_id).est) for user_id in users_not_buy_item]
    top_users = sorted(predicted_ratings, key=lambda x: x[1], reverse=True)[:k]
    
    recommended_items = []
    for user_id, _ in top_users:
        user_recommendations = get_recommendations(user_id, daily_sales_df, algo, k)
        recommended_items.extend(user_recommendations)

    return list(set(recommended_items))


In [19]:
# Функция для получения похожих товаров с использованием ансамбля моделей LightFM и Surprise
def ensemble_item_recommendations(item_id, daily_sales_df, lightfm_model, surprise_algo, k=5):
    lightfm_recs = get_similar_items(lightfm_model, item_id, k)
    surprise_recs = get_similar_items_surprise(item_id, daily_sales_df, surprise_algo, k)
    
    # Объединение рекомендаций от обеих моделей
    combined_recs = list(set(lightfm_recs + surprise_recs)) #['B00BUKL666', 'B01G0S3Y44', 'B00KSN9TME', 'B00DDT116M', 'B00R7PWK7W']
    
    lin = []
    for i in combined_recs:
        lin.append(daily_sales_df.loc[daily_sales_df['item'] == i]['title'].iloc[-1])
        
    
    result = list(filter(lambda x: x not in item_id, lin))
    
    return result[:k]

In [20]:
product_id = "B00MQ6F3AS"
product_recommendations = ensemble_item_recommendations(product_id, daily_sales_df, model, algo, k=5)

In [47]:
print(f"\nРекомендации для товара, ensemble_item_recommendatio:\n {daily_sales_df.loc[daily_sales_df['item'] == product_id]['title'].iloc[-1]}:\n")
for i in product_recommendations:
    print(i)


Рекомендации для товара, ensemble_item_recommendatio:
 Victor Allen's Coffee K Cups, Morning Blend Single Light Roast Coffee, 42 Count, Keurig 2.0 Brewer Compatible:

Premier Protein 30g Protein Shakes, Chocolate, 11 Fluid Ounces, 4 Per Pack
Victor Allen's Coffee K Cups, Decaf Donut Shop Blend Single Serve Medium Roast Coffee, 80 Count, Keurig 2.0 Brewer Compatible
Holiday Nuts Gift Basket - Gourmet Food Gifts Prime Delivery - Christmas, Mothers &amp; Fathers Day Fruit Nut Gift Box, Assortment Tray - Birthday, Sympathy, Get Well, Woman &amp; Families- Hula Delights
Newman's Own Organics Royal Tea, Organic Green Tea, 100 Tea Bags,7.1 ounce (Pack of 5)
International Delight, French Vanilla, Single-Serve Coffee Creamers, 24 Count (Pack of 6), Shelf Stable Non-Dairy Flavored Coffee Creamer, Great for Home Use, Offices, Parties or Group Events


# Агрегирование рейтингов:

In [22]:
#функция агрегирует рейтинги, предсказанные с помощью моделей LightFM и Surprise, и возвращает топ-N рекомендаций для заданного пользователя.
def ensemble_recommendations_by_rating(user_id, daily_sales_df, lightfm_model, surprise_algo, k=5):
    unique_products = daily_sales_df['item'].unique()
    
    # Получение рейтингов всех товаров для заданного пользователя с использованием моделей LightFM и Surprise
    lightfm_ratings = {product_id: lightfm_model.predict(np.array([dataset.mapping()[0][user_id]]), np.array([dataset.mapping()[2][product_id]]))[0] for product_id in unique_products}
    surprise_ratings = {product_id: surprise_algo.predict(user_id, product_id).est for product_id in unique_products}
    
    # Объединение рейтингов обеих моделей, усредняя их значения
    combined_ratings = {product_id: (lightfm_ratings[product_id] + surprise_ratings[product_id]) / 2 for product_id in unique_products}

    # Сортировка объединенных рейтингов в порядке убывания
    sorted_recommendations = sorted(combined_ratings.items(), key=lambda x: x[1], reverse=True)#[:k]
    
    lin = []
    for i in [product_id for product_id, _ in sorted_recommendations]: 
        lin.append(daily_sales_df.loc[daily_sales_df['item'] == i]['title'].iloc[-1])
        
    user_purchase = daily_sales_df.loc[daily_sales_df['user'] == user_id]['title'].unique().tolist()
    
    result = list(filter(lambda x: x not in user_purchase, lin))
    
    return result[:k]
 
#функция агрегирует рейтинги, предсказанные с помощью моделей LightFM и Surprise, но возвращает топ-N рекомендаций с учетом вероятности продажи товара. 
#Вероятность продажи используется для корректировки рейтинга каждого товара перед выбором топ-N.
def ensemble_recommendations_by_rating_adjusted(user_id, daily_sales_df, lightfm_model, surprise_algo, sales_probabilities, k=5):
    unique_products = daily_sales_df['item'].unique()
    
    # Получение рейтингов всех товаров для заданного пользователя с использованием моделей LightFM и Surprise
    user_idx = dataset.mapping()[0].get(user_id, -1)
    user_ids = np.full(len(unique_products), user_idx, dtype=np.int32)
    item_indices = [dataset.mapping()[2][product_id] for product_id in unique_products]

    lightfm_ratings = {product_id: rating for product_id, rating in zip(unique_products, lightfm_model.predict(user_ids, item_indices))}
    surprise_ratings = {product_id: surprise_algo.predict(user_id, product_id).est for product_id in unique_products}

    # Учитывание вероятности продажи товара для корректировки объединенных рейтингов
    adjusted_ratings = {}
    for product_id in unique_products:
        probability = sales_probabilities.loc[sales_probabilities['product_id'] == product_id, 'multiplier'].values[0]
        adjusted_ratings[product_id] = (lightfm_ratings[product_id] + surprise_ratings[product_id]) * probability

    # Сортировка скорректированных рейтингов в порядке убывания
    top_n_recommendations = sorted(adjusted_ratings, key=adjusted_ratings.get, reverse=True)#[:k]

    lin = []
    for i in top_n_recommendations: 
        lin.append(daily_sales_df.loc[daily_sales_df['item'] == i]['title'].iloc[-1])
        
    user_purchase = daily_sales_df.loc[daily_sales_df['user'] == user_id]['title'].unique().tolist()
    
    result = list(filter(lambda x: x not in user_purchase, lin))
    
    return result[:k]

In [23]:
# Используя агрегирование рейтингов
user_id = "A2PF64RBR1G1SZ"
recommendations_by_rating = ensemble_recommendations_by_rating(user_id, daily_sales_df, model, algo, k=5)

In [24]:
print(f"Рекомендации для пользователя купившего следующие товары, составленные ансамблем по методу 'Агрегирование рейтингов':\n") 
print('\n'.join(daily_sales_df.loc[daily_sales_df['user'] == user_id]['title'].unique().tolist()))

Рекомендации для пользователя купившего следующие товары, составленные ансамблем по методу 'Агрегирование рейтингов':

Twinings of London Pure Camomile Herbal Tea Bags, 20 Count
Yogi Tea - Peach DeTox - Healthy Warming Blend - 6 Pack, 96 Tea Bags Total
Twinings of London English Afternoon Black Tea Bags, 20 Count (Pack of 6)
Jack Link's Beef Jerky, Sweet &amp; Hot, 1.8-Ounce Packages (Pack of 8)
Davidson's Tea Irish Breakfast Tea, 100-Count Tea Bags
Bragg Organic Unfiltered Apple Cider Vinegar, Raw, 16 Ounce - 1 Pack
V8 Original 100% Vegetable Juice, 5.5 oz. Can (8 packs of 6, Total of 48)
Glycerin Vegetable - 1 Quart (43 oz.) - Non GMO - Sustainable Palm Based - USP - KOSHER - PURE - Pharmaceutical Grade
96-Count Beantown Roasters Variety Pack Sampler of 12 Assorted Roasted Coffees, single-cup coffee pack sampler for Keurig K-Cup Brewers
Twinings of London Christmas Black Tea Bags, 20 Count
Planters Mixed Nuts, Lightly Salted Deluxe Mixed Nuts, 15.25 Ounce
Stacy's Parmesan Garlic &amp

In [48]:
for i in recommendations_by_rating: print(i)

Perrier Lime Flavored Carbonated Mineral Water, 16.9 fl oz. Plastic Bottles (24 Count)
Hodgson Mill Honey Whole Wheat Bread Mix 16-Ounce Boxes (Pack of 6), Bread Mix for Bread Machines or Oven Baked Bread, Yeast Included
Frito-Lay Chips Classic Mix Multipack, 20 Count
Martinson Single Serve Coffee Capsules, Breakfast Blend, 24 Count
Bai Bubbles, Sparkling Water, Jamaica Blood Orange, Antioxidant Infused Drinks, 11.5  Fl Oz Cans, Pack of 12


In [26]:
# Используя дополненное агрегирование рейтингов
user_id = "A2PF64RBR1G1SZ"
recommendations_by_rating_adv = ensemble_recommendations_by_rating_adjusted(user_id, daily_sales_df, model, algo, multiplier_2002_full_df, k=5)

In [39]:
recommendations_by_rating_adv

['Perrier Lime Flavored Carbonated Mineral Water, 16.9 fl oz. Plastic Bottles (24 Count)',
 'Soylent Meal Replacement Drink, Original, 14 oz Bottles, 12 Pack (Packaging May Vary)',
 "Victor Allen's Coffee K Cups, Morning Blend Single Light Roast Coffee, 42 Count, Keurig 2.0 Brewer Compatible",
 'PBfit &mdash; All-Natural Peanut Butter Powder, Produced by BetterBody Foods, 8 Ounce',
 'IZZE Sparkling Juice, Blackberry, 8.4 oz Cans, 24 Count']

# Ранжирование рекомендаций:

In [28]:
#функция агрегирует рекомендации моделей LightFM и Surprise на основе ранга и возвращает топ-N рекомендаций для заданного пользователя.
def ensemble_recommendations_by_rank(user_id, daily_sales_df, lightfm_model, surprise_algo, k=5):
    unique_products = daily_sales_df['item'].unique()
    user_idx = dataset.mapping()[0][user_id]
    
    # Получение рейтингов всех товаров для заданного пользователя с использованием моделей LightFM и Surprise
    lightfm_ratings = {}
    for product_id in unique_products:
        product_idx = dataset.mapping()[2][product_id]
        lightfm_ratings[product_id] = lightfm_model.predict(user_idx, np.array([product_idx]))[0]
    
    surprise_ratings = {product_id: surprise_algo.predict(user_id, product_id).est for product_id in unique_products}
    
    # Ранжирование рекомендаций от каждой модели
    lightfm_ranked = sorted(lightfm_ratings.items(), key=lambda x: x[1], reverse=True)[:k]
    surprise_ranked = sorted(surprise_ratings.items(), key=lambda x: x[1], reverse=True)[:k]
    
    # Объединение рекомендаций на основе ранга
    recommendations = []
    for i in range(k):
        recommendations.append(lightfm_ranked[i][0])
        recommendations.append(surprise_ranked[i][0])

    lin = []
    for i in recommendations: 
        lin.append(daily_sales_df.loc[daily_sales_df['item'] == i]['title'].iloc[-1])
        
    user_purchase = daily_sales_df.loc[daily_sales_df['user'] == user_id]['title'].unique().tolist()
    
    result = list(filter(lambda x: x not in user_purchase, lin))
    
    return result[:k]

In [29]:
# Используя ранжирование рекомендаций
user_id = "A2PF64RBR1G1SZ"
ensemble_recommendations_rank = ensemble_recommendations_by_rank(user_id, daily_sales_df, model, algo, k=5)

In [30]:
print(f"Рекомендации для пользователя купившего следующие товары, составленные ансамблем по методу 'Ранжирование рекомендаций':\n") 
print('\n'.join(daily_sales_df.loc[daily_sales_df['user'] == user_id]['title'].unique().tolist()))

Рекомендации для пользователя купившего следующие товары, составленные ансамблем по методу 'Ранжирование рекомендаций':

Twinings of London Pure Camomile Herbal Tea Bags, 20 Count
Yogi Tea - Peach DeTox - Healthy Warming Blend - 6 Pack, 96 Tea Bags Total
Twinings of London English Afternoon Black Tea Bags, 20 Count (Pack of 6)
Jack Link's Beef Jerky, Sweet &amp; Hot, 1.8-Ounce Packages (Pack of 8)
Davidson's Tea Irish Breakfast Tea, 100-Count Tea Bags
Bragg Organic Unfiltered Apple Cider Vinegar, Raw, 16 Ounce - 1 Pack
V8 Original 100% Vegetable Juice, 5.5 oz. Can (8 packs of 6, Total of 48)
Glycerin Vegetable - 1 Quart (43 oz.) - Non GMO - Sustainable Palm Based - USP - KOSHER - PURE - Pharmaceutical Grade
96-Count Beantown Roasters Variety Pack Sampler of 12 Assorted Roasted Coffees, single-cup coffee pack sampler for Keurig K-Cup Brewers
Twinings of London Christmas Black Tea Bags, 20 Count
Planters Mixed Nuts, Lightly Salted Deluxe Mixed Nuts, 15.25 Ounce
Stacy's Parmesan Garlic &a

In [31]:
for i in ensemble_recommendations_rank:
    print(i)

Perrier Lime Flavored Carbonated Mineral Water, 16.9 fl oz. Plastic Bottles (24 Count)
Wilton 710-5506 Color Mist Spray 1.5oz, Black
NOW Foods Xylitol Natural Sweetener - 1 lb
Nutiva Organic, Cold-Pressed, Unrefined, Virgin Coconut Oil from Fresh, non-GMO, Sustainably Farmed Coconuts, 54-ounce
XyloSweet Non-GMO Xylitol Natural Sweetener, Granules, 5lb Resealable


In [32]:
#функция агрегирует рекомендации моделей LightFM и Surprise на основе дополненного ранга с учетом вероятности продажи товара. 
#Вероятность продажи используется для корректировки рейтинга каждого товара перед выбором топ-N рекомендаций.
def ensemble_recommendations_by_rank_adjusted(user_id, daily_sales_df, lightfm_model, surprise_algo, sales_probabilities, k=5):
    unique_products = daily_sales_df['item'].unique()
    
    user_idx = dataset.mapping()[0].get(user_id, -1)
    user_ids = np.full(len(unique_products), user_idx, dtype=np.int32)
    item_indices = [dataset.mapping()[2][product_id] for product_id in unique_products]

    # Получение рейтингов всех товаров для заданного пользователя с использованием моделей LightFM и Surprise
    lightfm_ratings = {product_id: rating for product_id, rating in zip(unique_products, lightfm_model.predict(user_ids, item_indices))}
    surprise_ratings = {product_id: surprise_algo.predict(user_id, product_id).est for product_id in unique_products}

    # Учитывание вероятности продажи товара для корректировки рейтингов    
    adjusted_lightfm_ratings = {}
    adjusted_surprise_ratings = {}
    for product_id in unique_products:
        probability = sales_probabilities.loc[sales_probabilities['product_id'] == product_id, 'multiplier'].values[0]
        adjusted_lightfm_ratings[product_id] = lightfm_ratings[product_id] * probability
        adjusted_surprise_ratings[product_id] = surprise_ratings[product_id] * probability

    # Ранжирование скорректированных рейтингов
    lightfm_ranked = sorted(adjusted_lightfm_ratings, key=adjusted_lightfm_ratings.get, reverse=True)
    surprise_ranked = sorted(adjusted_surprise_ratings, key=adjusted_surprise_ratings.get, reverse=True)

    # Объединение рекомендаций на основе дополненного ранга
    combined_ranked = rankdata(lightfm_ranked) + rankdata(surprise_ranked)
    recommendations = unique_products[np.argsort(-combined_ranked)]#[:k]
    
    lin = []
    for i in recommendations.tolist(): 
        lin.append(daily_sales_df.loc[daily_sales_df['item'] == i]['title'].iloc[-1])
        
    user_purchase = daily_sales_df.loc[daily_sales_df['user'] == user_id]['title'].unique().tolist()
    
    result = list(filter(lambda x: x not in user_purchase, lin))
    
    return result[:k]

In [33]:
# Используя дополненное ранжирование рекомендаций
user_id = "A2PF64RBR1G1SZ"
ensemble_recommendations_rank_adv = ensemble_recommendations_by_rank_adjusted(user_id, daily_sales_df, model, algo, multiplier_2002_full_df, k=5)

In [34]:
ensemble_recommendations_rank_adv

['Traditional Medicinals Organic Chamomile Herbal Leaf Tea, 16 Tea Bags (Pack of 6)',
 'Celestial Seasonings Victorian Earl Grey Tea, 25 Count',
 'Tim Hortons 100% Arabica Medium Roast Original Blend Ground Coffee, 32.8 Ounce',
 'Mentos Chewy Mint Candy Roll, Mint, Stocking Stuffer, Gift, Holiday, Christmas, Party, 1.32 Ounce/14 Pieces (Pack of 15) - Packaging May Vary',
 'Vanilla Bean Grass-Fed Ghee Butter by 4th &amp; Heart, 9 Ounce, Pasture Raised, Non-GMO, Lactose Free, Certified Paleo, Keto-Friendly']